In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from GameDayFunctions.fangraphs_projection_2020 import Projection
#from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer')

In [115]:
class Draft:

    def __init__(self, projections_object, 
                 draft_position = 6,
                 number_teams = 12,
                 roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5},
                 batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS'],
                 pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] ):

        self.number_rounds = sum(roster_spots.values())
        self.draft_position = draft_position
        self.round_number = 0
        self.draft_number = 0
        self.player_projections = projections_object
        self.remaining_ranked_players = projections_object.all_rank 
        #self.roto_standings = []
        self.roto_stats_batting = pd.DataFrame(columns =  batter_stats[1:])
        self.roto_stats_pitching = pd.DataFrame(columns =  pitcher_stats[1:])
        self.teams = {}
        for i in np.arange(number_teams):
            roto_stats = {}
            roto_stats['batting_stats'] = pd.DataFrame(columns =  batter_stats)
            roto_stats['pitching_stats'] = pd.DataFrame(columns =  pitcher_stats)
            roto_stats['roster_spots'] = roster_spots.copy()
            roto_stats['roster'] = {}
            self.teams[i] = roto_stats
    
    # Find Resulting Standings
    def tabulate_roto(self, teams):
        roto_stats_batting = self.roto_stats_batting.copy()
        roto_stats_pitching = self.roto_stats_pitching.copy()
        for iteam in np.arange(len(teams)):
            raw_team_batting = teams[iteam]['batting_stats']
            raw_team_pitching = teams[iteam]['pitching_stats']
            roto_team_batting = raw_team_batting.mean()
            roto_team_pitching = raw_team_pitching.mean()
            if 'AVG' in raw_team_batting:
                roto_team_batting['AVG'] = (raw_team_batting['AVG']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum()
            if 'OPS' in raw_team_batting:
                roto_team_batting['OPS'] = (raw_team_batting['OPS']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum() 
            if 'ERA' in raw_team_pitching:
                roto_team_batting['ERA'] = (raw_team_pitching['ERA']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum() 
            if 'WHIP' in raw_team_pitching:
                roto_team_batting['WHIP'] = (raw_team_pitching['WHIP']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum()
            batting_stat_names = self.roto_stats_batting.columns.values.tolist()
            pitching_stat_names = self.roto_stats_pitching.columns.values.tolist()
            roto_stats_batting = roto_stats_batting.append(roto_team_batting[batting_stat_names],ignore_index = True)
            roto_stats_pitching = roto_stats_pitching.append(roto_team_pitching[pitching_stat_names],ignore_index = True)
            
        roto_team_stats = pd.concat([roto_stats_batting, roto_stats_pitching], axis=1, sort=False)
        roto_standings_avgs = pd.concat([roto_stats_batting.rank(ascending=False), roto_stats_pitching.rank(ascending=False).rename(columns={"BB": "BBP"})], axis=1, sort=False)
        roto_standings_cnts = pd.concat([roto_stats_batting.rank(), roto_stats_pitching.rank().rename(columns={"BB": "BBP"})], axis=1, sort=False)
        avg_stats = ['CS','BBP','ERA','WHIP','BSV']
        for avg_stat in avg_stats:
            if avg_stat in roto_standings_avgs:
                roto_standings_cnts[avg_stat] = roto_standings_avgs[avg_stat]
        roto_standings = roto_standings_cnts.sum(axis=1).sort_values(ascending=False)
        roto_placement = roto_standings.index.get_loc(self.draft_position)
        #pdb.set_trace()
        return roto_stats_batting, roto_stats_pitching, roto_standings, roto_placement
     
    def confirm_selection(self, teams, team_key, selected_player):
        # Check if Position is a Pitcher or Batter
        if selected_player.EligiblePosition.str.contains('P').bool() == True:
            idx_player = self.player_projections.pitchers_stats.Name.values == selected_player.iloc[0].PLAYER
            statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
            teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

        else:
            idx_player = self.player_projections.hitters_stats.Name.values == selected_player.iloc[0].PLAYER
            statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
            teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)
        return teams
    
    # Do the entire draft one round at a time
    def draft_all(self):
        for iround in np.arange(self.number_rounds):
            self.draft_round(iround)
            self.round_number += 1
        pdb.set_trace()
        self.roto_stats_batting, self.roto_stats_pitching, self.roto_standings, self.roto_placement = self.tabulate_roto(self.teams)
      
    def draft_remaining(self, teams):
        for iround in range(self.round_number,self.number_rounds):
            #self.draft_round(iround)
            
            draft_order = np.arange(len(self.teams))
            # Reverse draft order every other round
            if iround % 2 == 1:
                draft_order = draft_order[::-1]
            
            if iround == self.round_number:
                draft_order = draft_order[self.draft_position + 1:] # wont work if draw first or last picks...
                
            for iteam in draft_order:
                teams = self.draft_next_best(iteam, teams)

        return teams
        
    
    def draft_round(self, round_key):
        
        # Reverse draft order every other round
        draft_order = np.arange(len(self.teams))
        if round_key % 2 == 1:
            draft_order = draft_order[::-1]
        
        # Reset draft_number at start of round
        self.draft_number = len(self.teams) - len(draft_order)
        
        # Draft each round one team at a time
        for iteam in draft_order:
            
            # When team is draft_position, search for best pick.
            if iteam == self.draft_position:
                best_pick = self.find_best_pick(iteam)
                self.teams, self.remaining_ranked_players = self.draft_next_best(iteam, force_pick = best_pick)
            else: 
                self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            self.draft_number += 1
    
    # Define logic for round selection
    def best_pick = find_best_pick(self, team_key):
        # best_pick is iloc of df to be used in draft_next_best
        
        df = self.remaining_ranked_players
        
        # Find best player at each remaining position
        unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]
        ups = '|'.join(unfilled_positions)
        print(ups)  
        
        # Set next best player first.        
        idx_eligible = [0]        
        #pdb.set_trace()
        # WANT TO PUT GOOD PLAYERS IN UTIL, BUT NOT BN
        for iunfilled in unfilled_positions:
            if (iunfilled != 'UTIL') & (iunfilled != 'BN'):
                idx_position = [i for i, val in enumerate(df.EligiblePosition.str.contains(iunfilled)) if val] 
                filled = False
                jdx = 0
                while filled == False:
                    if idx_position[jdx] in idx_eligible:
                        jdx+=1
                    else:
                        idx_eligible.append(idx_position[jdx])
                        filled = True
        
        # Get rid of doubles (1B and OF is particularly prone)
        idx_eligible = np.unique(idx_eligible)
        print('Picking from:')
        print(df.iloc[idx_eligible])
        #pdb.set_trace()
        
        #################################
        # START OF LOOP TO FIND BEST PLAYER
        player_based_draft_outcomes = {}

        # Loop over eligible players, then finish the draft
        #for iposition in idx_eligible:
        for iposition, icounter in zip(idx_eligible, range(len(idx_eligible))):
            pick_ok = False
            
            # make a copy of teams to finish drafting
            teams = self.teams.copy()

            # Check if next best player spot is taken
            is_eligibile = False
            check_eligible = df.iloc[idx_eligible[icounter]].EligiblePosition.split('/')
            for icheck in check_eligible:
                if self.teams[team_key]['roster_spots'][icheck] > 0:
                    is_elegible = True
            
            #pdb.set_trace()
            # If the first value is UTIL or BN 
            if is_elegible == False:

                # Check if there are UTIL positions left
                df_copy,drafted_player=df.drop(df.iloc[0:1].index),df.iloc[0:1]
                eligible_positions = drafted_player.EligiblePosition.values[0]
                if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                    if self.teams[team_key]['roster_spots']['P'] > 0:
                        position = 'P'
                        df = df_copy
                    elif self.teams[team_key]['roster_spots']['BN'] > 0:
                        position = 'BN'
                        df = df_copy
                    else:
                        position = 'No Roster Spots Left'
                        pick_ok = False
                else:
                    if self.teams[team_key]['roster_spots']['UTIL'] > 0:
                        position = 'UTIL'
                        df = df_copy
                    elif self.teams[team_key]['roster_spots']['BN'] > 0:
                        position = 'BN'
                        df = df_copy
                    else:
                        position = 'No Roster Spots Left'
                        pick_ok = False

                #print(self.teams[team_key]['roster_spots'])
                #print('Considering '+drafted_player.PLAYER+' for '+position)
            else:
                pick = idx_eligible[0]
                df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
                eligible_positions = drafted_player.EligiblePosition.values[0]

                # For those eligible for multiple positions, split them up and pick one
                tf_position = [p in eligible_positions for p in unfilled_positions]
                idx_position = [i for i, val in enumerate(tf_position) if val] 
                position = unfilled_positions[idx_position[0]]
                pick_ok = True

            if pick_ok == True:
                #teams = self.confirm_selection(teams, team_key, selected_player)
                # Check if Position is a Pitcher or Batter
                if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                    idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                    statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                    teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

                else:
                    idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                    statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                    teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)

            # LOOP OVER WHOLE REST OF THE DRAFT HERE...
            pdb.set_trace()
            teams = self.draft_remaining(teams)
            
            # Calculate the best pseudo-standings
            pseudo_standings = self.tabulate_roto(teams)
            
            # Decide best choice
            # LOGIC...
            
            # Store the result.  
            #player_based_draft_outcomes = {idx_player:}
            
        # Decide on best choice and return 
        best_pick = 
        
        # END OF LOOP TO FIND BEST PLAYER
        #################################
        ## Remove player from self.remaining_ranked_players 
        #self.remaining_ranked_players = df

        # Eliminate one roster_spot
        #self.teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        #if position in self.teams[team_key]['roster']:
        #    self.teams[team_key]['roster'][position] = [self.teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        #else:
        #    self.teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
                #print('Team '+ str(team_key) +' Drafting '+drafted_player.PLAYER+' for '+position)
            #else:
            #    print('Not Drafting '+drafted_player.PLAYER)
            #    print(ups)
            #pdb.set_trace()
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best(self,team_key, teams = self.teams, df = self.remaining_ranked_players, force_pick = 0):
        if force_pick == 0:
            pick_ok = False
            
            while pick_ok == False:
                pick_ok = True

                # Figure out what positions are still unfilled.
                unfilled_positions = [k for (k,v) in teams[team_key]['roster_spots'].items() if v > 0]
                ups = '|'.join(unfilled_positions)
                #print(ups)

                # Find indices of all remaining players in positions of need.
                idx_eligible = [i for i, val in enumerate(df.EligiblePosition.str.contains(ups)) if val]

                # If the next best draft pick's position is filled, see if possible to be UTIL or BN 
                if len(idx_eligible) == 0 or idx_eligible[0] != 0:
                    # Check if there are UTIL positions left
                    df_copy,drafted_player=df.drop(df.iloc[0:1].index),df.iloc[0:1]
                    eligible_positions = drafted_player.EligiblePosition.values[0]
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        if teams[team_key]['roster_spots']['P'] > 0:
                            position = 'P'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False
                    else:
                        if teams[team_key]['roster_spots']['UTIL'] > 0:
                            position = 'UTIL'
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False

                else:
                    pick = idx_eligible[0]
                    df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
                    eligible_positions = drafted_player.EligiblePosition.values[0]

                    # For those eligible for multiple positions, split them up and pick one
                    tf_position = [p in eligible_positions for p in unfilled_positions]
                    idx_position = [i for i, val in enumerate(tf_position) if val] 
                    position = unfilled_positions[idx_position[0]]

                if pick_ok == True:
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        self.teams[team_key]['pitching_stats'] = self.teams[team_key]['pitching_stats'].append(statline)

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        self.teams[team_key]['batting_stats'] = self.teams[team_key]['batting_stats'].append(statline)

        else:
            df,drafted_player=df.drop(df.iloc[force_pick:force_pick+1].index),df.iloc[force_pick:force_pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]

            # For those eligible for multiple positions, split them up and pick one
            tf_position = [p in eligible_positions for p in unfilled_positions]
            idx_position = [i for i, val in enumerate(tf_position) if val] 
            position = unfilled_positions[idx_position[0]]
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline)

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline)        # Remove player from self.remaining_ranked_players 
        
        print('About to remove from remaing_ranked_players')
        print('Team '+ str(team_key) +' Drafting '+drafted_player.PLAYER+' for '+position)
        pdb.set_trace()
        #remaining_ranked_players = df

        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [self.teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df

                    

In [119]:
x = range(1,6)
x2 = np.arange(1,6)
for n in x2:
  print(n)

1
2
3
4
5


In [122]:
draft_order = np.arange(12)
#draft_order = draft_order[::-1]
c = 6
for n in draft_order[c:]:
  print(n)

6
7
8
9
10
11


In [116]:
zips = Draft(player_ranking_zips, number_teams = 12)
steam = Draft(player_ranking_steam, number_teams = 12)

In [117]:
zips.draft_all()

C|1B|2B|3B|SS|OF|UTIL|SP|RP|P|BN
Picking from:
                PLAYER EligiblePosition
Rank                                   
7       Cody Bellinger            1B/OF
185      J.T. Realmuto                C
8      Freddie Freeman               1B
15     Whit Merrifield            2B/OF
9         Alex Bregman            3B/SS
14    Francisco Lindor               SS
12       J.D. Martinez               OF
11    Justin Verlander               SP
27       Roberto Osuna               RP
13        Jacob deGrom               SP
> <ipython-input-115-12dc68ae7342>(183)draft_pick()
-> pseudo_standings = self.tabulate_roto(teams)


(Pdb)  teams


{0: {'batting_stats':         AB   R  1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
11477  539  99  91  34  4  34  106  24  77  0.302  0.964, 'pitching_stats': Empty DataFrame
Columns: [IP, W, L, CG, SHO, SV, BB, SO, ERA, WHIP, BSV]
Index: [], 'roster_spots': {'C': 1, '1B': 1, '2B': 1, '3B': 1, 'SS': 1, 'OF': 2, 'UTIL': 1, 'SP': 2, 'RP': 2, 'P': 3, 'BN': 5}, 'roster': {'OF': 'Christian Yelich'}}, 1: {'batting_stats':         AB    R   1B  2B 3B  HR RBI  SB  BB    AVG    OPS
13611  608  126  101  39  5  34  97  21  79  0.294  0.918, 'pitching_stats': Empty DataFrame
Columns: [IP, W, L, CG, SHO, SV, BB, SO, ERA, WHIP, BSV]
Index: [], 'roster_spots': {'C': 1, '1B': 1, '2B': 1, '3B': 1, 'SS': 1, 'OF': 2, 'UTIL': 1, 'SP': 2, 'RP': 2, 'P': 3, 'BN': 5}, 'roster': {'OF': 'Mookie Betts'}}, 2: {'batting_stats':         AB    R  1B  2B 3B  HR  RBI  SB   BB    AVG    OPS
10155  472  107  70  26  3  40  102  17  112  0.294  1.055, 'pitching_stats': Empty DataFrame
Columns: [IP, W, L, CG, SHO, SV, BB, SO

(Pdb)  exit


BdbQuit: 

In [7]:
steam.draft_all()

In [8]:
steam.roto_placement

9

In [8]:
zips.roto_standings

0     171.0
9     163.0
7     156.0
1     144.5
10    143.0
4     123.0
6     117.0
11    114.5
3     114.0
8     108.0
2     103.5
5     102.5
dtype: float64

In [9]:
steam.roto_standings

0     180.0
9     162.0
10    153.0
11    140.5
1     130.0
4     125.0
7     122.0
3     122.0
2     116.5
6     115.0
5     114.0
8      80.0
dtype: float64

In [10]:
steam.teams[6]['roster']

{'1B': 'Cody Bellinger',
 'SS': 'Fernando Tatis',
 '2B': 'Ozzie Albies',
 'RP': ['Aroldis Chapman', 'Tyler Duffey'],
 'UTIL': 'Carlos Santana',
 'SP': ['Drew Pomeranz', 'Brandon Woodruff'],
 'P': [['Zack Britton', 'John Brebbia'], 'Colin Poche'],
 'BN': [[[['Yoan López', 'Keynan Middleton'], 'Mark Melancon'], 'Luis Patiño'],
  'Tyler Rogers'],
 '3B': 'Luis Arraez',
 'OF': [['Nick Senzel', 'Willie Calhoun'], 'Franmil Reyes'],
 'C': 'Carson Kelly'}